In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import BernoulliRBM
from scipy.ndimage import convolve
from sklearn import linear_model, datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import clone
%matplotlib inline

In [19]:
df = pd.read_csv('fashion-mnist_test.csv')


# Challenge

 Use RBM to perform feature extraction on an image-based dataset that you find or create. If you go this route, present the features you extract and explain why this is a useful feature extraction method in the context you’re operating in. DO NOT USE either the MNIST digit recognition database or the iris data set. They’ve been worked on in very public ways very very many times and the code is easily available. (However, that code could be a useful resource to refer to). _OR_,

In [20]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
X = df.loc[:, "pixel1":]
Y = df['label']

In [34]:
from sklearn.preprocessing import Binarizer


binarizer = Binarizer(threshold=0.0).fit(X)
binaryX = binarizer.transform(X)
X = binaryX

In [35]:
dataframe = pd.DataFrame.from_records(X)

In [37]:
X = dataframe

In [40]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.1, random_state=0)

# Models we will use
logistic = LogisticRegression(solver='lbfgs', max_iter=10000,
                              C=100, multi_class='multinomial')

rbm = BernoulliRBM(n_iter=20, n_components=500, random_state=42, 
                   learning_rate=.1, verbose=True)

rbm_features_classifier = Pipeline(
    steps=[('rbm', rbm), ('logistic', logistic)])

rbm_features_classifier.fit(X_train, Y_train)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -266.76, time = 3.88s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -218.97, time = 4.34s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -199.44, time = 4.26s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -196.34, time = 4.26s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -185.19, time = 4.25s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -184.73, time = 4.24s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -178.83, time = 4.24s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -190.70, time = 4.24s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -180.77, time = 4.28s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -178.81, time = 4.23s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -207.71, time = 4.36s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -179.61, time = 4.28s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -181.69, time = 4.23s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -181.92, time = 4.25s
[BernoulliRBM] 

Pipeline(memory=None,
     steps=[('rbm', BernoulliRBM(batch_size=10, learning_rate=0.1, n_components=500, n_iter=20,
       random_state=42, verbose=True)), ('logistic', LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

In [41]:
raw_pixel_classifier = clone(logistic)
raw_pixel_classifier.C = 100.
raw_pixel_classifier.fit(X_train, Y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [42]:
Y_pred = rbm_features_classifier.predict(X_test)
print("Logistic regression using RBM features:\n%s\n" % (
    metrics.classification_report(Y_test, Y_pred)))

Y_pred = raw_pixel_classifier.predict(X_test)
print("Logistic regression using raw pixel features:\n%s\n" % (
    metrics.classification_report(Y_test, Y_pred)))

Logistic regression using RBM features:
              precision    recall  f1-score   support

           0       0.77      0.75      0.76        88
           1       0.98      0.93      0.95        86
           2       0.45      0.66      0.54       112
           3       0.79      0.82      0.81        94
           4       0.44      0.44      0.44       110
           5       0.95      0.92      0.93       108
           6       0.44      0.28      0.34       104
           7       0.90      0.81      0.86       113
           8       0.88      0.83      0.85        99
           9       0.77      0.87      0.82        86

   micro avg       0.72      0.72      0.72      1000
   macro avg       0.74      0.73      0.73      1000
weighted avg       0.73      0.72      0.72      1000


Logistic regression using raw pixel features:
              precision    recall  f1-score   support

           0       0.62      0.66      0.64        88
           1       0.93      0.92      0.92  